In [ ]:
class TextCNN_temp(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    <Parameters>
        - sequence_length: 최대 문장 길이
        - num_classes: 클래스 개수
        - embedding_size: 각 단어에 해당되는 임베디드 벡터의 차원
        - filter_sizes: convolutional filter들의 사이즈 (= 각 filter가 몇 개의 단어를 볼 것인가?) (예: "3, 4, 5")
        - num_filters: 각 filter size 별 filter 수
        - l2_reg_lambda: 각 weights, biases에 대한 l2 regularization 정도
    """
    def make_filter_set(self, filter_size, num_filters):
        if type(filter_size) == int:
            filter_size = [filter_size]
        if type(num_filters) == int:
            num_filters = [num_filters]

        filter_first = True
        if len(filter_size) >= len(num_filters):
            t1 = filter_size
            t2 = num_filters
        else:
            t1 = num_filters
            t2 = filter_size
            filter_first = False

        result_arr = []

        for i, t in enumerate(t1):
            j = i

            if i >= len(t2):
                j = i%len(t2)

            if filter_first:
                c = (t , t2[j])
            else:
                c = (t2[j], t)

            result_arr.append(c)

        return result_arr
        
    def __init__(
            self, sequence_length, num_classes, 
            embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0, conv_horizon_concat=True):
       
        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.float32, [None, sequence_length, embedding_size], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)

        self.expended_input_x = tf.expand_dims(self.input_x, -1) #conv2를 위한 차원 expend
        test_set = self.make_filter_set(filter_size=filter_sizes,num_filters=num_filters)
        
        # Create a convolution + maxpool layer for each filter size
        num_filters_total = 0
        input_vec = None
        pooled_outputs = []
        pre_filter = 1
        
        for idx, filter_set in enumerate(test_set):
            filter_size, num_filter = filter_set
            with tf.name_scope("conv-%s" % filter_size):
                if input_vec is None or conv_horizon_concat:
                    input_vec = self.expended_input_x
                    pre_filter = 1
                
                sentence_length = int(input_vec.shape[1])
                word_vector_size = int(input_vec.shape[2])
                
                # Convolution Layer
                filter_shape = [filter_size, word_vector_size, pre_filter, num_filter]
                print("filter shape")
                print(filter_shape)
                W = tf.Variable(tf.truncated_normal(filter_shape), name="W") #변경 가능 영역 - stddev / truncated_normal<->random_normal
                b = tf.Variable(tf.random_normal(shape=[num_filter]), name="b")
                conv = tf.nn.conv2d(
                    input_vec,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                print("conv result shape")
                print(h.shape)
                
            with tf.name_scope("maxpool-%s" % filter_size):
                # Maxpooling over the outputs
#                 sentence_length = int(h.shape[1])
                word_vector_size = int(h.shape[2])
                
                whole_size = sentence_length - filter_size
                if whole_size < 0:
                    whole_size = 0
                        
                if conv_horizon_concat:
                    pool_size = [1, whole_size + 1, 1, 1]
                else:   
                    #변경가능 영역
#                     vec_size = int(word_vector_size/len(test_set*2))
#                     if vec_size == 0:
#                         vec_size = 1
                    vec_size = 1

                    str_size = int((whole_size)/len(test_set))
                    if str_size == 0:
                        str_size = 1
#                     str_size = filter_size
                        
                    pool_size = [1,str_size,vec_size,1] #조정가능
                    
                    if idx+1 == len(test_set):#고정영역                        
                        pool_size = [1, whole_size + 1, word_vector_size, 1]
                print("pool size")
                print(pool_size)
                pooled = tf.nn.max_pool(
                    h,
                    ksize=pool_size,
                    strides=[1,1,1,1],
                    padding='VALID',
                    name="pool")
                print("pooled shape")
                print(pooled.shape)
                if conv_horizon_concat:
                    pooled_outputs.append(pooled)
                    num_filters_total += num_filter
                else:
#                     with tf.name_scope("dropout-%s" % filter_size):
#                         p_drop = tf.nn.dropout(pooled, self.dropout_keep_prob)
                    input_vec = pooled
                    pre_filter = num_filter
                    num_filters_total = num_filter

        # Add dropout
        if conv_horizon_concat:
            # Combine all the pooled features
            self.h_pool = tf.concat(axis=3, values=pooled_outputs)
        else:
            self.h_pool = input_vec
        
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)
            
        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        # Calculate Mean cross-entropy loss
        with tf.name_scope("loss"):
            losses =tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.input_y, logits=self.scores)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

In [ ]:
class TextCNN_vertical_noDrop(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    <Parameters>
        - sequence_length: 최대 문장 길이
        - num_classes: 클래스 개수
        - embedding_size: 각 단어에 해당되는 임베디드 벡터의 차원
        - filter_sizes: convolutional filter들의 사이즈 (= 각 filter가 몇 개의 단어를 볼 것인가?) (예: "3, 4, 5")
        - num_filters: 각 filter size 별 filter 수
        - l2_reg_lambda: 각 weights, biases에 대한 l2 regularization 정도
    """

    def __init__(
            self, sequence_length, num_classes, 
            embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):
       
        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.float32, [None, sequence_length, embedding_size], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)

        self.expended_input_x = tf.expand_dims(self.input_x, -1) #conv2를 위한 차원 expend

        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        test_set = make_filter_set(filter_size=filter_sizes,num_filters=num_filters)
        
        num_filters_total = 0
        input_vec = None
        pre_filter = 1
        for idx, filter_set in enumerate(test_set):
            filter_size, num_filter = filter_set
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                if input_vec is None:
                    input_vec = self.expended_input_x
  
                # Convolution Layer
                filter_shape = [filter_size, int(input_vec.shape[2]), pre_filter, num_filter]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filter]), name="b")
                conv = tf.nn.conv2d(
                    input_vec,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                # Maxpooling over the outputs
                pool_size = [1,int((int(input_vec.shape[1]) - filter_size)/len(test_set)),1,1] #조정가능
                
                if idx+1 == len(test_set):
                    pool_size = [1, int(input_vec.shape[1]) - filter_size + 1, 1, 1]
                pooled = tf.nn.max_pool(
                    h,
                    ksize=pool_size,
#                     ksize=[1, int(input_vec.shape[1]) - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                input_vec = pooled
                pre_filter = num_filter
#                 pooled_outputs.append(pooled)
                
                num_filters_total = num_filter
            
        # Combine all the pooled features
#         num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = input_vec#tf.concat(axis=3, values=pooled_outputs)
        
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])

        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        # Calculate Mean cross-entropy loss
        with tf.name_scope("loss"):
            losses =tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.input_y, logits=self.scores)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")